In [ ]:
# to load and link kernal in notebook
!conda activate text_to_3d_env_llama3
!pip install notebook
!pip install ipykernel
!python -m ipykernel install --user --name=mykernal

In [3]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes wandb
!pip install torch --index-url https://download.pytorch.org/whl/cu121

## SETUP MODEL

In [9]:
import unsloth
from unsloth import FastLanguageModel

ImportError: Unsloth: Please import Unsloth before bitsandbytes.

In [6]:
from unsloth import FastLanguageModel

version = "v6"
max_seq_length = 32768 #32768 #2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
output_dir = f"arjuntheprogrammer/llama3-8B-racks-v6{version}"

# from transformers import AutoModel, AutoTokenizer
# model_id = "arjuntheprogrammer/llama3-8B-racks-v6"
# model = AutoModel.from_pretrained(model_id)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = output_dir, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

ImportError: Unsloth: Please import Unsloth before bitsandbytes.

## INFERENCE

In [4]:
prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

input_text = """Create a 3D racks cluster system with the following specifications:

- The cluster consists of 3 row and 2 column with a distance of 200 units between rows and 300 units between columns.
- Inside the cluster, there is a rack with 3 row and 1 column.
- The rack contains trays where pallets are placed. Each pallet has an offset of x: 12, y: 0, z: 11 units.

Start building the cluster system based on these dimensions and arrangements.

Following are the different assets with their dimensions in JSON format which you can use to build the 3D environment:
{
  "SM_Box_2": {"id": 0, "dimensions": [42, 90, 50]},
  "SM_Box_3": {"id": 1, "dimensions": [20, 40, 20]},
  "SM_Box_4": {"id": 2, "dimensions": [20, 20, 20]},
  "SM_Box_5": {"id": 3, "dimensions": [40, 40, 40]},
  "SM_Box_6": {"id": 4, "dimensions": [40, 40, 20]},
  "SM_Box_7": {"id": 5, "dimensions": [40, 80, 20]},
  "SM_Box_8": {"id": 6, "dimensions": [42, 90, 30]},
  "SM_Pallet": {"id": 7, "dimensions": [121, 101, 12]},
  "SM_Rack_Frame": {"id": 8, "dimensions": [10, 100, 128]},
  "SM_Rack_Tray": {"id": 9, "dimensions": [288, 100, 16]}
}
"""

inputs = tokenizer(
[
    prompt_template.format(
        input_text,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs, streamer = text_streamer,
    max_new_tokens = max_seq_length,
)

ModuleNotFoundError: No module named 'bitsandbytes'